# 💻 Geokoding i geopandas


Geopandas støtter geokoding via et bibliotek kalt
[geopy](http://geopy.readthedocs.io/), som må være installert for å bruke
[geopandas ' `geopandas.tools.geocode()`
funksjon](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html).
`geocode()` forventer en `liste` eller `pandas.Series` av adresser (strenger) og
returnerer en `GeoDataFrame` med løste adresser og punktgeometrier.

La oss prøve dette ut.

Vi vil geokode adresser lagret i en semikolon-separert tekstfil kalt
`adresser.txt`. Disse adressene ligger i alle i Oslo.

In [1]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [2]:
import pandas
adresser = pandas.read_csv(
    DATA_MAPPE / "oslo_adresser" / "adresser.txt",
    sep=";"
)

adresser.head()

,id,adr
0,100,"Statsråd Mathiesens vei 25, 0594 OSLO"
1,101,"Slimeveien 15, 1275 OSLO"
2,102,"Sognsveien 80, 0855 OSLO"
3,103,"Ullevålsveien 5, 0165 OSLO"
4,104,"Nydalsveien 30b, 0484 OSLO"


Vi har en `id` for hver rad og en adresse i `adr` kolonnen.


## Geokode adresser ved hjelp av *Nominatim*

I vårt eksempel vil vi bruke *Nominatim* som en *geokodingstilbyder*. [*Nominatim*](https://nominatim.org/) er et bibliotek og en tjeneste som bruker OpenStreetMap-data, og drives av OpenStreetMap Foundation. Geopandas '
[`geocode()`
funksjon](https://geopandas.org/en/stable/docs/reference/api/geopandas.tools.geocode.html) støtter den innebygd.

```{admonition} God bruk
:class: note

[Nominatims brukervilkår](https://operations.osmfoundation.org/policies/nominatim/)
krever at brukere av tjenesten sørger for at de ikke sender mer hyppige
forespørsler enn en per sekund, og at en tilpasset **bruker-agent** streng er
knyttet til hver forespørsel.

Geopandas' implementering lar oss spesifisere en `user_agent`; biblioteket tar også
hånd om å respektere hastighetsbegrensningen til Nominatim.

Å slå opp en adresse er en ganske kostbar databaseoperasjon. Derfor er det,
noen ganger, den offentlige og gratise Nominatim-serveren bruker litt lenger tid å på
svare. I dette eksempelet legger vi til en parameter `timeout=10` for å vente opptil 10
sekunder for et svar.
```

In [3]:
import geopandas

geokodede_adresser = geopandas.tools.geocode(
    adresser["adr"],
    provider="nominatim",
    user_agent="gmgi221",
    timeout=10
)
geokodede_adresser.head()

KeyboardInterrupt: 

Et voilà! Som et resultat fikk vi tilbake en `GeoDataFrame` som inneholder en analysert
versjon av våre originale adresser og en `geometry` kolonne med
`shapely.geometry.Point`s som vi kan bruke, for eksempel, til å eksportere dataene til
et romlig dataformat.

Imidlertid ble `id`-kolonnen forkastet i prosessen. For å kombinere inputdatasettet med resultatsettet vårt, kan vi bruke pandas' [*join*
operasjoner](https://pandas.pydata.org/docs/user_guide/merging.html).


## Koble sammen dataframes

:::{admonition} Koble sammen datasett ved hjelp av pandas
:class: note

For en omfattende oversikt over forskjellige måter å kombinere dataframes og
Series, ta en titt på pandas dokumentasjon om [merge,
join og
concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).
:::


Å koble data fra to eller flere dataframes eller tabeller er en vanlig oppgave i mange
(romlige) dataanalysearbeidsflyter. Som du kanskje husker fra våre tidligere
timer, kan kombinering av data fra forskjellige tabeller basert på en felles **nøkkel**-attributt
gjøres enkelt i pandas/geopandas ved hjelp av [`merge()`
funksjonen](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Men, noen ganger er det nyttig å koble to dataframes sammen basert på deres
**indeks**. Dataframes må ha **samme antall rader** og
**dele den samme indeksen** (enkelt forklart, de skal ha samme rekkefølge av rader).

Vi kan bruke denne tilnærmingen, for å koble informasjon fra den originale dataframen `adresser` til de geokodede adressene `geokodede_adresser`, rad for rad.
`join()`-funksjonen, som standard, kobler to dataframes basert på indeksen deres.
Dette fungerer for eksemplet vårt, da rekkefølgen på de to dataframesene er identiske.

In [4]:
geokodede_adresser_med_id = geokodede_adresser.join(adresser)
geokodede_adresser_med_id

,geometry,address,id,adr
0,POINT (10.83648 59.94104),"25, Statsråd Mathiesens vei, Linderud, Bjerke,...",100,"Statsråd Mathiesens vei 25, 0594 OSLO"
1,POINT (10.83432 59.83557),"15, Slimeveien, Bjørnholt, Søndre Nordstrand, ...",101,"Slimeveien 15, 1275 OSLO"
2,POINT (10.72956 59.95011),"Sognsveien 80, Konvallveien, Sogn, Nordre Aker...",102,"Sognsveien 80, 0855 OSLO"
3,POINT (10.74356 59.91863),"5, Ullevålsveien, Hammersborg, St. Hanshaugen,...",103,"Ullevålsveien 5, 0165 OSLO"
4,POINT (10.76402 59.9503),"30B, Nydalsveien, Nydalen, Nordre Aker, Oslo, ...",104,"Nydalsveien 30b, 0484 OSLO"
5,POINT (10.75292 59.919),"3, Vestre Elvebakke, Fredensborg, Grünerløkka,...",105,"Vestre Elvebakke 3, 0182 OSLO"
6,POINT (10.79645 59.90968),"5, Etterstadsletta, Gamle Oslo, Oslo, 0660, Norge",106,"Etterstadsletta 5, 0660 OSLO"
7,POINT (10.75544 59.92704),"20, Steenstrups gate, Grünerløkka, Oslo, 0554,...",107,"Steenstrups gate 20, 0554 OSLO"
8,POINT (10.79432 59.91526),"21, Fyrstikkalléen, Lilleberg, Gamle Oslo, Osl...",108,"Fyrstikkalleen 21, 0661 OSLO"
9,POINT (10.71724 59.91854),"Niels Juels gate, Uranienborg, Frogner, Oslo, ...",109,"Niels Juels gate 56, 0259 OSLO"


Utdataen fra `join()` er en ny `geopandas.GeoDataFrame`:

In [5]:
type(geokodede_adresser_med_id)

geopandas.geodataframe.GeoDataFrame

Den nye data rammen har alle originale kolonner pluss nye kolonner for `geometry`
og for en analysert `adresse` som kan brukes til å spot-sjekke resultatene.

:::{note}
Hvis du skulle gjøre join den andre veien, dvs. `adresser.join(geokodede_adresser)`, ville utdata være en `pandas.DataFrame`, ikke en `geopandas.GeoDataFrame`.
:::


---


Det er nå enkelt å lagre det nye datasettet som en geospatial fil, for eksempel, i
*GeoPackage* format:

In [6]:
# slett en muligens eksisterende fil, da den skaper
# problemer i tilfelle sphinx kjøres gjentatte ganger
try:
    (DATA_MAPPE / "oslo_adresser" / "adresser.gpkg").unlink()
except FileNotFoundError:
    pass

In [7]:
geokodede_adresser_med_id.to_file(DATA_MAPPE / "oslo_adresser" / "adresser.gpkg")